In [1]:
import pandas as pd
import numpy as np
import scipy as sp
from scipy.stats import uniform
from os import listdir, getcwd
import re
import sys
import warnings
import sklearn
import pickle
import openpyxl

### IMPORT DATASET 
##### ONLY LMCI/EMCI SUBJECTS; 
##### BASELINE INFORMATION + CONVERSION AT 3-YEARS 

In [4]:
dataset = pd.read_excel("../adnimerge_screen.xlsx")

### CREATE PREDICTORS AND OUTCOME ARRAYS, AND SPLIT THEM IN TRAIN AND TEST DATASETS

In [11]:
# The outcome variable is CONVERSION_AT_3Y, the last column
# The site variable is at index 3

X = dataset.iloc[:,0:(dataset.shape[1]-1)].as_matrix()
y = dataset["CONVERSION_AT_3Y"].as_matrix()

num_cases = X.shape[0]

SITES = np.unique(X[:,3])

count = 0
save_set = pd.DataFrame([[0]], columns=["split"])
while count < 10000:
    SITES_this = SITES.copy()
    SITE_test = np.random.choice(SITES_this,1)
    SITES_this = SITES_this[[i for i in range(SITES_this.shape[0]) if SITES_this[i] not in SITE_test]]
    # random choise of
    while X[[i for i in range(X.shape[0]) if X[i,3] in SITE_test],:].shape[0]/num_cases < 0.19:
        SITE_test = np.append(SITE_test,np.random.choice(SITES_this,1))
        SITES_this = SITES_this[[i for i in range(SITES_this.shape[0]) if SITES_this[i] not in SITE_test]]
    
    if X[[i for i in range(X.shape[0]) if X[i,3] in SITE_test],:].shape[0]/num_cases < 0.21: 
        
        y_train = y[[i for i in range(X.shape[0]) if X[i,3] not in SITE_test]]
        y_test = y[[i for i in range(X.shape[0]) if X[i,3] in SITE_test]]

        if np.abs((y_train[y_train == 1].shape[0]/y_train.shape[0]) - (y_test[y_test == 1].shape[0]/y_test.shape[0])) < 0.003:
            
            print("____________________________")
            print("____________________________")
            print("____________________________")
            print("____________________________")
            print("____________________________")
                            
            print(count)
            print(np.sort(SITE_test))
            
            print("% of cases hold-out for the test set is:")
            print(y_train.shape[0]/y.shape[0])

            print("____________________________")

            print("N cases in train set: "+str(y_train.shape[0]))
            print("% of CONVERTERS in the train set is: "+str(y_train[y_train == 1].shape[0]/y_train.shape[0]))
            print("% of NON-CONVERTERS in the train set is: "+str(y_train[y_train == 0].shape[0]/y_train.shape[0]))


            print("N cases in test set: "+str(y_test.shape[0]))
            print("% of CONVERTERS in the train set is: "+str(y_test[y_test == 1].shape[0]/y_test.shape[0]))
            print("% of NON-CONVERTERS in the train set is: "+str(y_test[y_test == 0].shape[0]/y_test.shape[0]))
            
            save_set = save_set.append(pd.Series(np.sort(SITE_test)), ignore_index=True)
            
    
    count += 1

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  This is separate from the ipykernel package so we can avoid doing imports until


____________________________
____________________________
____________________________
____________________________
____________________________
45
[  14.   21.   22.   27.   94.  130.  133.  136.  137.]
% of cases hold-out for the test set is:
0.8018181818181818
____________________________
N cases in train set: 441
% of CONVERTERS in the train set is: 0.35827664399092973
% of NON-CONVERTERS in the train set is: 0.6417233560090703
N cases in test set: 109
% of CONVERTERS in the train set is: 0.3577981651376147
% of NON-CONVERTERS in the train set is: 0.6422018348623854
____________________________
____________________________
____________________________
____________________________
____________________________
97
[   7.   11.   18.   22.   32.   35.  109.  114.  128.  129.  136.]
% of cases hold-out for the test set is:
0.7981818181818182
____________________________
N cases in train set: 439
% of CONVERTERS in the train set is: 0.357630979498861
% of NON-CONVERTERS in the train set 

KeyboardInterrupt: 

### TRYING TO MAKE A SERIES OF FOLDS THAT WILL COVER THE ENTIRE DATASET

In [12]:
# cleanse the dataframe

if save_set.shape[0] > 1:
    save_set = save_set.drop("split",axis=1).drop(0,axis=0)
    # print(save_set)

In [13]:
# IF IT EXISTS, LOAD PREVIOUSLY SAVED SPLITS AND ADD THE NEW ONES

if "test_splits.xlsx" in listdir(getcwd()):
    save_set_previous = pd.read_excel("test_splits.xlsx")
    if save_set.shape[0] > 1:
        save_set = save_set_previous.append(save_set)
    else:
        save_set = save_set_previous
        
    # REMOVE DUPLCATES, IF ANY
    save_set.index = range(save_set.shape[0])
    duplicated = np.where(save_set.duplicated(subset=None, keep='first') == True)[0]
    if len(duplicated)>0:
        save_set = save_set.drop(duplicated, axis=0)
        
save_set.index = range(save_set.shape[0])

In [17]:
# save the obtained test split adding this the the previously found ones

save_set.to_excel("test_splits.xlsx")
# print(save_set)

In [ ]:
# TRYing TO LOOK FOR INDEPENDENT SPLIT SETS AMONG THOSE IDENTIFIED WITH THE PREVIOSU RANDOM SEARCH

pd.set_option('display.max_columns', 50)

######## LEVEL 1

for i_1 in np.array(save_set.index):
    
    print("PERCENT OF PROGRESSION: " + str(     round(((i_1+1)/save_set.shape[0])*100,1)      )+"%")
    
    save_set_minus_i_1 = save_set.drop(i_1, axis=0)
    save_set_minus_i_1.index = range(save_set_minus_i_1.shape[0])
    
    ALL_CENTERS_BY_NOW_1 = np.array(save_set.iloc[i_1])[np.where(~(np.isnan(save_set.iloc[i_1])))[0]]
    ALL_CENTERS_BY_NOW_dataframe_1 = pd.DataFrame(np.array(save_set.iloc[i_1]).reshape((1,save_set.shape[1])))
    
    y_train = y[[i for i in range(X.shape[0]) if X[i,int(np.array(np.where(np.array(predictor_names) == "SITE")))] not in ALL_CENTERS_BY_NOW_1]]
    
    if y_train.shape[0]/y.shape[0] < 0.775 or y_train.shape[0]/y.shape[0] > 0.825:
        continue
    
############# LEVEL 2
    
    for i_2 in np.array(save_set_minus_i_1.index)[i_1:len(save_set_minus_i_1)]:
        
        save_set_minus_i_1_this = np.array(save_set_minus_i_1.iloc[i_2])
        
        save_set_minus_i_1_this = save_set_minus_i_1_this[np.where(~(np.isnan(save_set_minus_i_1_this)))[0]]
        
        count_j_2 = 0
        for j_2 in range(len(save_set_minus_i_1_this)):
            if save_set_minus_i_1_this[j_2] in ALL_CENTERS_BY_NOW_1:
                count_j_2 += 1
                break
        
        if count_j_2 > 0:
            continue
            
        y_train = y[[i for i in range(X.shape[0]) if X[i,int(np.array(np.where(np.array(predictor_names) == "SITE")))] not in save_set_minus_i_1_this]]
    
        if y_train.shape[0]/y.shape[0] < 0.775 or y_train.shape[0]/y.shape[0] > 0.825:
             continue
        
        ALL_CENTERS_BY_NOW_2 = np.append(ALL_CENTERS_BY_NOW_1, np.array(save_set_minus_i_1_this)).copy()
        ALL_CENTERS_BY_NOW_dataframe_2 = ALL_CENTERS_BY_NOW_dataframe_1.append(pd.Series(save_set_minus_i_1_this), ignore_index=True).copy()
        
        save_set_minus_i_2 = save_set_minus_i_1.drop(i_2, axis=0)
        save_set_minus_i_2.index = range(save_set_minus_i_2.shape[0])

        
#################### LEVEL 3
            
        for i_3 in np.array(save_set_minus_i_2.index)[i_2:len(save_set_minus_i_2)]:
            
            
            save_set_minus_i_2_this = np.array(save_set_minus_i_2.iloc[i_3])
                
            save_set_minus_i_2_this = save_set_minus_i_2_this[np.where(~(np.isnan(save_set_minus_i_2_this)))[0]] 
                        
            count_j_3 = 0    
            for j_3 in range(len(save_set_minus_i_2_this)):
                if save_set_minus_i_2_this[j_3] in ALL_CENTERS_BY_NOW_2:
                    count_j_3 += 1
                    break
            
            if count_j_3 > 0:
                continue
                
            y_train = y[[i for i in range(X.shape[0]) if X[i,int(np.array(np.where(np.array(predictor_names) == "SITE")))] not in save_set_minus_i_2_this]]
    
            if y_train.shape[0]/y.shape[0] < 0.775 or y_train.shape[0]/y.shape[0] > 0.825:
                continue
                        
            ALL_CENTERS_BY_NOW_3 = np.append(ALL_CENTERS_BY_NOW_2, np.array(save_set_minus_i_2_this)).copy()
            ALL_CENTERS_BY_NOW_dataframe_3 = ALL_CENTERS_BY_NOW_dataframe_2.append(pd.Series(save_set_minus_i_2_this), ignore_index=True).copy()
            
            save_set_minus_i_3 = save_set_minus_i_2.drop(i_3, axis=0)
            save_set_minus_i_3.index = range(save_set_minus_i_3.shape[0]) 
                  
            
                
                
############################# LEVEL 4               
            
            for i_4 in np.array(save_set_minus_i_3.index)[i_3:len(save_set_minus_i_3)]:
            
                save_set_minus_i_3_this = np.array(save_set_minus_i_3.iloc[i_4])
                
                save_set_minus_i_3_this = save_set_minus_i_3_this[np.where(~(np.isnan(save_set_minus_i_3_this)))[0]]
            
                count_j_4 = 0    
                for j_4 in range(len(save_set_minus_i_3_this)):
                    if save_set_minus_i_3_this[j_4] in ALL_CENTERS_BY_NOW_3:
                        count_j_4 += 1
                        break
            
                if count_j_4 > 0:
                    continue
                    
                y_train = y[[i for i in range(X.shape[0]) if X[i,int(np.array(np.where(np.array(predictor_names) == "SITE")))] not in save_set_minus_i_3_this]]
    
                if y_train.shape[0]/y.shape[0] < 0.775 or y_train.shape[0]/y.shape[0] > 0.825:
                    continue
                        
                ALL_CENTERS_BY_NOW_4 = np.append(ALL_CENTERS_BY_NOW_3, np.array(save_set_minus_i_3_this)).copy()
                ALL_CENTERS_BY_NOW_dataframe_4 = ALL_CENTERS_BY_NOW_dataframe_3.append(pd.Series(save_set_minus_i_3_this), ignore_index=True).copy()
            
                save_set_minus_i_4 = save_set_minus_i_3.drop(i_4, axis=0)
                save_set_minus_i_4.index = range(save_set_minus_i_4.shape[0]) 
                
############################# LEVEL 5    
                
                save_set_minus_i_4_this = np.array(SITES[[i for i in range(len(SITES)) if SITES[i] not in np.sort(ALL_CENTERS_BY_NOW_4)]])
            
                y_train = y[[i for i in range(X.shape[0]) if X[i,int(np.array(np.where(np.array(predictor_names) == "SITE")))] not in save_set_minus_i_4_this]]
    
                if y_train.shape[0]/y.shape[0] < 0.775 or y_train.shape[0]/y.shape[0] > 0.825:
                    continue
            
                ALL_CENTERS_BY_NOW_5 = np.append(ALL_CENTERS_BY_NOW_4, np.array(SITES[[i for i in range(len(SITES)) if SITES[i] not in np.sort(ALL_CENTERS_BY_NOW_4)]])).copy()
                ALL_CENTERS_BY_NOW_dataframe_5 = ALL_CENTERS_BY_NOW_dataframe_4.append(pd.Series(SITES[[i for i in range(len(SITES)) if SITES[i] not in np.sort(ALL_CENTERS_BY_NOW_4)]]), ignore_index=True).copy()
            
                display(ALL_CENTERS_BY_NOW_dataframe_5)
                

PERCENT OF PROGRESSION: 0.2%
PERCENT OF PROGRESSION: 0.3%
PERCENT OF PROGRESSION: 0.5%
PERCENT OF PROGRESSION: 0.7%
PERCENT OF PROGRESSION: 0.8%
PERCENT OF PROGRESSION: 1.0%
PERCENT OF PROGRESSION: 1.2%
PERCENT OF PROGRESSION: 1.4%
PERCENT OF PROGRESSION: 1.5%
PERCENT OF PROGRESSION: 1.7%
PERCENT OF PROGRESSION: 1.9%
PERCENT OF PROGRESSION: 2.0%
PERCENT OF PROGRESSION: 2.2%
PERCENT OF PROGRESSION: 2.4%
PERCENT OF PROGRESSION: 2.5%
PERCENT OF PROGRESSION: 2.7%
PERCENT OF PROGRESSION: 2.9%
PERCENT OF PROGRESSION: 3.1%
PERCENT OF PROGRESSION: 3.2%
PERCENT OF PROGRESSION: 3.4%
PERCENT OF PROGRESSION: 3.6%
PERCENT OF PROGRESSION: 3.7%
PERCENT OF PROGRESSION: 3.9%


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
0,2.0,5.0,11.0,32.0,33.0,41.0,67.0,73.0,98.0,109.0,136.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3.0,7.0,13.0,16.0,24.0,35.0,51.0,128.0,137.0,153.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,12.0,18.0,23.0,37.0,53.0,57.0,72.0,99.0,130.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,9.0,21.0,27.0,36.0,68.0,94.0,100.0,116.0,141.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,6.0,10.0,14.0,19.0,22.0,29.0,31.0,52.0,62.0,82.0,114.0,123.0,126.0,127.0,129.0,131.0,133.0,135.0,941.0


PERCENT OF PROGRESSION: 4.1%
PERCENT OF PROGRESSION: 4.2%
PERCENT OF PROGRESSION: 4.4%


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,14.0,23.0,27.0,29.0,62.0,72.0,94.0,99.0,100.0,123.0,941.0,NaN,NaN,NaN,NaN,NaN
1,3.0,16.0,36.0,41.0,51.0,52.0,68.0,98.0,109.0,127.0,129.0,136.0,NaN,NaN,NaN,NaN
2,13.0,22.0,31.0,32.0,33.0,37.0,126.0,128.0,141.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2.0,5.0,7.0,9.0,18.0,19.0,35.0,67.0,73.0,116.0,135.0,NaN,NaN,NaN,NaN,NaN
4,6.0,10.0,11.0,12.0,21.0,24.0,53.0,57.0,82.0,114.0,130.0,131.0,133.0,137.0,153.0,NaN


PERCENT OF PROGRESSION: 4.6%
PERCENT OF PROGRESSION: 4.8%


...

PERCENT OF PROGRESSION: 29.9%
PERCENT OF PROGRESSION: 30.1%
PERCENT OF PROGRESSION: 30.2%


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,5.0,6.0,7.0,12.0,18.0,22.0,53.0,72.0,116.0,127.0,131.0,NaN,NaN,NaN,NaN,NaN
1,16.0,27.0,33.0,37.0,57.0,114.0,128.0,129.0,153.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,19.0,21.0,35.0,36.0,52.0,94.0,98.0,100.0,130.0,137.0,NaN,NaN,NaN,NaN,NaN,NaN
3,9.0,11.0,24.0,29.0,41.0,51.0,62.0,68.0,73.0,123.0,126.0,136.0,NaN,NaN,NaN,NaN
4,2.0,3.0,10.0,13.0,14.0,23.0,31.0,32.0,67.0,82.0,99.0,109.0,133.0,135.0,141.0,941.0


PERCENT OF PROGRESSION: 30.4%
PERCENT OF PROGRESSION: 30.6%
PERCENT OF PROGRESSION: 30.7%
PERCENT OF PROGRESSION: 30.9%
PERCENT OF PROGRESSION: 31.1%
PERCENT OF PROGRESSION: 31.2%
PERCENT OF PROGRESSION: 31.4%
PERCENT OF PROGRESSION: 31.6%
PERCENT OF PROGRESSION: 31.7%
PERCENT OF PROGRESSION: 31.9%
PERCENT OF PROGRESSION: 32.1%
PERCENT OF PROGRESSION: 32.3%
PERCENT OF PROGRESSION: 32.4%
PERCENT OF PROGRESSION: 32.6%
PERCENT OF PROGRESSION: 32.8%
PERCENT OF PROGRESSION: 32.9%
PERCENT OF PROGRESSION: 33.1%
PERCENT OF PROGRESSION: 33.3%
PERCENT OF PROGRESSION: 33.4%
PERCENT OF PROGRESSION: 33.6%
PERCENT OF PROGRESSION: 33.8%
PERCENT OF PROGRESSION: 34.0%
PERCENT OF PROGRESSION: 34.1%
PERCENT OF PROGRESSION: 34.3%
PERCENT OF PROGRESSION: 34.5%
PERCENT OF PROGRESSION: 34.6%
PERCENT OF PROGRESSION: 34.8%
PERCENT OF PROGRESSION: 35.0%
PERCENT OF PROGRESSION: 35.1%
PERCENT OF PROGRESSION: 35.3%
PERCENT OF PROGRESSION: 35.5%
PERCENT OF PROGRESSION: 35.7%
PERCENT OF PROGRESSION: 35.8%


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,24.0,29.0,32.0,33.0,68.0,116.0,123.0,130.0,131.0,135.0,137.0,153.0,NaN,NaN,NaN,NaN
1,5.0,7.0,53.0,94.0,100.0,109.0,126.0,128.0,141.0,941.0,NaN,NaN,NaN,NaN,NaN,NaN
2,9.0,13.0,16.0,19.0,27.0,41.0,52.0,57.0,73.0,127.0,129.0,NaN,NaN,NaN,NaN,NaN
3,21.0,22.0,23.0,31.0,35.0,37.0,82.0,98.0,99.0,114.0,NaN,NaN,NaN,NaN,NaN,NaN
4,2.0,3.0,6.0,10.0,11.0,12.0,14.0,18.0,36.0,51.0,62.0,67.0,72.0,133.0,136.0,NaN


PERCENT OF PROGRESSION: 36.0%
PERCENT OF PROGRESSION: 36.2%
PERCENT OF PROGRESSION: 36.3%
PERCENT OF PROGRESSION: 36.5%
PERCENT OF PROGRESSION: 36.7%
PERCENT OF PROGRESSION: 36.8%
PERCENT OF PROGRESSION: 37.0%
PERCENT OF PROGRESSION: 37.2%
PERCENT OF PROGRESSION: 37.4%
PERCENT OF PROGRESSION: 37.5%
PERCENT OF PROGRESSION: 37.7%


# TEST THE 5-FOLD CROSS-CENTER SPLITS

In [78]:
##### FOLD 1

OUTER_FOLD_1 = [6.0,12.0,18.0,21.0,126.0,127.0,128.0,137.0]

y_test = y[[i for i in range(X.shape[0]) if X[i,int(np.array(np.where(np.array(predictor_names) == "SITE")))] in OUTER_FOLD_1]]                    
            
print("OUTER_FOLD_1")
print("   ")
print("% of cases: "+ str(round(y_test.shape[0]/y.shape[0]*100,2))+"%")
print("N of cases: "+str(y_test.shape[0]))
print("   ")
print("% of CONVERTERS: "+str(round(y_test[y_test == 1].shape[0]/y_test.shape[0]*100,2))+"%")
print("% of NON-CONVERTERS: "+str(round(y_test[y_test == 0].shape[0]/y_test.shape[0]*100,2))+"%")

OUTER_FOLD_1
   
% of cases: 20.91%
N of cases: 115
   
% of CONVERTERS: 35.65%
% of NON-CONVERTERS: 64.35%


In [79]:
##### FOLD 2

OUTER_FOLD_2 = [2.0,3.0,9.0,23.0,24.0,29.0,36.0,37.0,94.0,99.0,114.0]

y_test = y[[i for i in range(X.shape[0]) if X[i,int(np.array(np.where(np.array(predictor_names) == "SITE")))] in OUTER_FOLD_2]]                    
            
print("OUTER_FOLD_2")
print("   ")
print("% of cases: "+ str(round(y_test.shape[0]/y.shape[0]*100,2))+"%")
print("N of cases: "+str(y_test.shape[0]))
print("   ")
print("% of CONVERTERS: "+str(round(y_test[y_test == 1].shape[0]/y_test.shape[0]*100,2))+"%")
print("% of NON-CONVERTERS: "+str(round(y_test[y_test == 0].shape[0]/y_test.shape[0]*100,2))+"%")

OUTER_FOLD_2
   
% of cases: 19.82%
N of cases: 109
   
% of CONVERTERS: 35.78%
% of NON-CONVERTERS: 64.22%


In [84]:
##### FOLD 3

OUTER_FOLD_3 = [7.0,13.0,14.0,33.0,41.0,67.0,73.0,98.0,100.0,109.0,116.0]

y_test = y[[i for i in range(X.shape[0]) if X[i,int(np.array(np.where(np.array(predictor_names) == "SITE")))] in OUTER_FOLD_3]]                    
            
print("OUTER_FOLD_3")
print("   ")
print("% of cases: "+ str(round(y_test.shape[0]/y.shape[0]*100,2))+"%")
print("N of cases: "+str(y_test.shape[0]))
print("   ")
print("% of CONVERTERS: "+str(round(y_test[y_test == 1].shape[0]/y_test.shape[0]*100,2))+"%")
print("% of NON-CONVERTERS: "+str(round(y_test[y_test == 0].shape[0]/y_test.shape[0]*100,2))+"%")

OUTER_FOLD_3
   
% of cases: 19.82%
N of cases: 109
   
% of CONVERTERS: 35.78%
% of NON-CONVERTERS: 64.22%


In [85]:
##### FOLD 4

OUTER_FOLD_4 = [5.0,16.0,22.0,27.0,31.0,35.0,123.0,130.0,141.0,153.0]

y_test = y[[i for i in range(X.shape[0]) if X[i,int(np.array(np.where(np.array(predictor_names) == "SITE")))] in OUTER_FOLD_4]]                    
            
print("OUTER_FOLD_4")
print("   ")
print("% of cases: "+ str(round(y_test.shape[0]/y.shape[0]*100,2))+"%")
print("N of cases: "+str(y_test.shape[0]))
print("   ")
print("% of CONVERTERS: "+str(round(y_test[y_test == 1].shape[0]/y_test.shape[0]*100,2))+"%")
print("% of NON-CONVERTERS: "+str(round(y_test[y_test == 0].shape[0]/y_test.shape[0]*100,2))+"%")

OUTER_FOLD_4
   
% of cases: 19.82%
N of cases: 109
   
% of CONVERTERS: 35.78%
% of NON-CONVERTERS: 64.22%


In [83]:
##### FOLD 5

OUTER_FOLD_5 = [10.0,11.0,19.0,32.0,51.0,52.0,53.0,57.0,62.0,68.0,72.0,82.0,129.0,131.0,133.0,135.0,136.0,941.0]

y_test = y[[i for i in range(X.shape[0]) if X[i,int(np.array(np.where(np.array(predictor_names) == "SITE")))] in OUTER_FOLD_5]]                    
            
print("OUTER_FOLD_5")
print("   ")
print("% of cases: "+ str(round(y_test.shape[0]/y.shape[0]*100,2))+"%")
print("N of cases: "+str(y_test.shape[0]))
print("   ")
print("% of CONVERTERS: "+str(round(y_test[y_test == 1].shape[0]/y_test.shape[0]*100,2))+"%")
print("% of NON-CONVERTERS: "+str(round(y_test[y_test == 0].shape[0]/y_test.shape[0]*100,2))+"%")

OUTER_FOLD_5
   
% of cases: 19.64%
N of cases: 108
   
% of CONVERTERS: 36.11%
% of NON-CONVERTERS: 63.89%


In [100]:
####### CHECK IF THE 5-FOLD SPLIT CONTAINS ALL THE SITES

ALL_SITES_FROM_SPLIT = np.sort(np.append(np.append(np.append(np.append(np.array(OUTER_FOLD_1), np.array(OUTER_FOLD_2)), np.array(OUTER_FOLD_3)), np.array(OUTER_FOLD_4)), np.array(OUTER_FOLD_5) ))

ALL_SITES_FROM_SPLIT == SITES

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True])